In [1]:
# 查看当前kernel下的package
# !pip list --format=columns

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import warnings
import gc
import os
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
from collections import OrderedDict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
import time
from itertools import combinations

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

In [3]:
train_df=pd.read_csv('/home/mw/input/pre8881/train.csv')
test_df=pd.read_csv('/home/mw/input/pretest_a3048/test_a.csv')

In [4]:
# print(train_df.head(5))

In [5]:
# 合并train test的特征 进行特征预处理
c_train=train_df.copy()
c_test=test_df.copy()
print(c_train.shape)
print(c_test.shape)

(684283, 65)
(186925, 64)


In [6]:
c_train['train']=1
c_test['train']=0
#c_train=c_train.drop(['y1_is_purchase'],axis=1)
df=pd.concat([c_train,c_test],axis=0,sort=False)

In [7]:
# 1 查看每一列缺失比例
NAN=[[c,df[c].isna().mean()*100] for c in df.columns]
NAN=pd.DataFrame(NAN,columns=['col_name','percent'])
NAN=NAN.sort_values('percent',ascending=False)
# print(NAN)

In [8]:
# 删除空值比例太多的列 暂时50%
NAN=NAN[NAN['percent']>50]
col_nan=list(NAN['col_name'])
print(col_nan)
df=df.drop(col_nan,axis=1)

['num_notcar_claim', 'dur_personal_insurance_90', 'p2_is_smeowner', 'p2_is_enterprise_owner', 'p2_marital_status']


In [9]:
# 挑选分类列和数值列
object_columns_df=df.select_dtypes(include=['object'])
numerical_columns_df=df.select_dtypes(exclude=['object'])
# print(object_columns_df.dtypes)
# print(numerical_columns_df.dtypes)
null_counts=object_columns_df.isnull().sum()
print('number of null values in each col\n：{}'.format(null_counts))

number of null values in each col
：client_no                        9435
xz                                  0
xb                                  0
carid                               0
regdate                             0
trademark_cn                        0
brand_cn                            0
make_cn                             0
series                             58
use_type                            0
change_owner                        0
p1_gender                        9448
p1_census_register              10754
f1_child_flag                   47818
f2_posses_house_flag           105898
p2_client_grade                 23034
w1_pc_wx_use_flag                9441
p1_is_bank_eff                  24455
p2_is_child_under_15_family      9456
p2_is_adult_over_55_family       9454
birth_month                     52129
dtype: int64


In [10]:
# 2 填失空值-分类型col
columns_fill_none=['regdate','brand_cn','make_cn','series','p1_census_register','f1_child_flag',\
'f2_posses_house_flag','p2_client_grade','w1_pc_wx_use_flag','birth_month']
object_columns_df[columns_fill_none]=object_columns_df[columns_fill_none].fillna('None')

In [11]:
columns_fill_max=['xz', 'use_type', 'change_owner', 'p1_is_bank_eff', 'p1_gender',\
'p2_is_child_under_15_family', 'p2_is_adult_over_55_family',]
object_columns_df[columns_fill_max]=object_columns_df[columns_fill_max].fillna(object_columns_df.mode().iloc[0])

In [12]:
# null_counts=object_columns_df.isnull().sum()
# print('number of null values in each col\n：{}'.format(null_counts))

In [13]:
# print(list(object_columns_df.columns))

In [14]:
# # 2 填失空值-数值型列
# null_counts=numerical_columns_df.isnull().sum()
# print('number of null values in each col\n：{}'.format(null_counts))

In [15]:
# 平均值填充一部分 其余用0填充
columns_fill_ave = ['p1_age', 'f2_cust_housing_price_total']
for c in columns_fill_ave:
    numerical_columns_df[c] = numerical_columns_df[c].fillna(
        numerical_columns_df[c].mean())
numerical_columns_df = numerical_columns_df.fillna(0)


In [16]:
# null_counts=numerical_columns_df.isnull().sum()
# print('number of null values in each col\n：{}'.format(null_counts))

In [17]:
del c_train,c_test,train_df,test_df
gc.collect()

7

In [18]:
# 分类特征 one-hot或者 有序
# bin_map={'':}
# one_hot_columns=[ 'xz', 'xb', 'trademark_cn', \
# 'brand_cn',  'use_type', 'change_owner', 'p1_gender', \
# 'p1_census_register', 'f1_child_flag', 'f2_posses_house_flag', 'p2_client_grade',\
#  'w1_pc_wx_use_flag', 'p1_is_bank_eff', 'p2_is_child_under_15_family', \
#  'p2_is_adult_over_55_family', 'birth_month']
for f in list(object_columns_df.columns):
    if f in ['carid', 'regdate']:
        continue
    le = LabelEncoder()
    object_columns_df[f] = le.fit_transform(
        object_columns_df[f].astype('str')).astype('int')


In [19]:
# 分开
df_final=pd.concat([object_columns_df,numerical_columns_df],axis=1,sort=False)
drop_cols=['client_no','regdate','make_cn', 'series','use_type','num_notcar_claim',\
'p2_is_child_under_15_family','suiche_nonauto_amount_18','p2_is_adult_over_55_family',\
'p2_is_smeowner','suiche_nonauto_nprem_18','p2_is_enterprise_owner']
#df_final = df_final.drop(drop_cols,axis=1)
print('ok')

ok


### 构造新特征

In [20]:
df_final['tp_ratio'] = df_final['nprem_tp'] / df_final['si_tp']
# 计数
for f in [['dpt'], ['client_no'], ['trademark_cn'], ['brand_cn'], ['make_cn'], ['series']]:
    df_temp = df_final.groupby(f).size().reset_index()
    df_temp.columns = f + ['{}_count'.format('_'.join(f))]
    df_final = df_final.merge(df_temp, how='left')


In [21]:
# print(df_final['birth_month'].head())
# df_final['birth_month'] = df_final['birth_month'].apply(
#     lambda x: int(x[:-1]) if type(x) != float else 0)

In [22]:
print(df_final.head())

   client_no  xz  xb                                         carid  \
0     103123   2   0  WY4N+MOjfIx8wJ3j6GhlA4qEfL71brEUkqbB0SSdqkI=   
1     728448   2   0  DXMuODygH0ddFea7SIoAOhF4134Bx4TPvkkPi6WCdzU=   
2     797949   2   0  waWCEYZJqj9PYxFdVeVLkpCNf/n0BdXPFi1iHlk0WWk=   
3     847824   2   0  nyRm/VviYGDpy2errRWE206SaYkVuqeclusAtXEU9v8=   
4      11727   0   5  LacSDMaoqD0AJRqCeYaGUu343r4NQiVuiFc9hyjLcMI=   

               regdate  trademark_cn  brand_cn  make_cn  series  use_type  \
0  2010-02-09 00:00:00           150       505     8816     651         1   
1  2007-02-02 00:00:00            12       188     1893     930         1   
2  2007-01-01 00:00:00           189       497    10291     647         1   
3  2015-02-09 00:00:00            74         7     5666     137         1   
4  2017-12-12 00:00:00            61        18     4735    1091         1   

   change_owner  p1_gender  p1_census_register  f1_child_flag  \
0             1          0                 180     

In [23]:
# 简单统计
def stat(df, df_merge, group_by, agg):
    group = df.groupby(group_by).agg(agg)

    columns = []
    for on, methods in agg.items():
        for method in methods:
            columns.append('{}_{}_{}'.format('_'.join(group_by), on, method))
    group.columns = columns
    group.reset_index(inplace=True)
    df_merge = df_merge.merge(group, on=group_by, how='left')

    del (group)
    gc.collect()

    return df_merge


def statis_feat(df_know, df_unknow):
    for f in tqdm(['p1_census_register', 'dpt']):
        df_unknow = stat(df_know, df_unknow, [f], {
                         'y1_is_purchase': ['mean']})

    return df_unknow

In [24]:
# 5折交叉
df_train = df_final[df_final['train']==1]
df_train = df_train.reset_index(drop=True)
df_test = df_final[df_final['train'] == 0]

seed=2021
df_stas_feat = None
kfold = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
for train_index, val_index in kfold.split(df_train, df_train['y1_is_purchase']):
    df_fold_train = df_train.iloc[train_index]
    df_fold_val = df_train.iloc[val_index]

    df_fold_val = statis_feat(df_fold_train, df_fold_val)
    df_stas_feat = pd.concat([df_stas_feat, df_fold_val], axis=0)

    del(df_fold_train)
    del(df_fold_val)
    gc.collect()

df_test = statis_feat(df_train, df_test)
df_final = pd.concat([df_stas_feat, df_test], axis=0)

del(df_stas_feat)
del(df_train)
del(df_test)
gc.collect()

100%|██████████| 2/2 [00:00<00:00,  4.95it/s]


36

## 模型

In [34]:
# drop_cols=['train','client_no','regdate','make_cn', 'series','use_type','num_notcar_claim',\
# 'p2_is_child_under_15_family','suiche_nonauto_amount_18','p2_is_adult_over_55_family',\
# 'p2_is_smeowner','suiche_nonauto_nprem_18','p2_is_enterprise_owner']
drop_cols=['train','client_no','p2_is_enterprise_owner','use_type','num_notcar_claim',\
'p2_is_child_under_15_family']
# drop_cols=['train','p2_is_enterprise_owner','use_type','num_notcar_claim',\
# 'p2_is_child_under_15_family']
df_train = df_final[df_final['train']==1]
df_test = df_final[df_final['train']==0]

In [35]:
ycol = 'y1_is_purchase'
feature_names = list(
    filter(lambda x: x not in [ycol, 'regdate', 'carid']+drop_cols, df_train.columns))

model = lgb.LGBMClassifier(num_leaves=16,
                           max_depth=4,
                           learning_rate=0.01,
                           n_estimators=1000,
                           subsample=0.8,
                           feature_fraction=0.8,
                           reg_alpha=0.5,
                           reg_lambda=0.5,
                           random_state=seed,
                           metric=None)

oof = []
prediction = df_test[['carid']]
prediction['label'] = 0
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(
        df_train[feature_names], df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    lgb_model = model.fit(X_train,
                          Y_train,
                          eval_names=['valid'],
                          eval_set=[(X_val, Y_val)],
                          verbose=100,
                          eval_metric='auc',
                          early_stopping_rounds=10)

    pred_val = lgb_model.predict_proba(
        X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
    df_oof = df_train.iloc[val_idx][[
        'carid', ycol]].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = lgb_model.predict_proba(
        df_test[feature_names], num_iteration=lgb_model.best_iteration_)[:, 1]
    prediction['label'] += pred_test / 5

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.804596
[200]	valid's auc: 0.808843
[300]	valid's auc: 0.812405
[400]	valid's auc: 0.81529
[500]	valid's auc: 0.817305


In [ ]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()
df_importance

In [28]:
df_oof = pd.concat(oof)
score = roc_auc_score(df_oof['y1_is_purchase'], df_oof['pred'])

In [29]:
score

In [ ]:
os.makedirs('sub', exist_ok=True)
prediction.to_csv(f'sub/{score}.csv', index=False)
prediction.to_csv(f'sub/sub.csv', index=False)

In [ ]:
# 模型融合